<a href="https://colab.research.google.com/github/nontivillu/nontivillu/blob/main/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEEK 6 Big Data Best Practices

In [1]:
# Step 1: Install
!pip install openpyxl pyspark

In [2]:
## Step 2: Import the pandas and spark libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col

In [3]:
# Step 3: Download the Online Retail Dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx -O online_retail.xlsx

# Step 3: Convert the XLSX File to CSV for Easier Processing with Spark
excel_data = pd.read_excel('online_retail.xlsx', engine='openpyxl')
excel_data.to_csv('online_retail.csv', index=False)

--2024-11-16 14:27:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘online_retail.xlsx’

online_retail.xlsx      [     <=>            ]  22.62M  22.6MB/s    in 1.0s    

2024-11-16 14:27:59 (22.6 MB/s) - ‘online_retail.xlsx’ saved [23715344]



In [4]:
# Step 4: Initialize SparkSession
spark = SparkSession.builder.appName("OnlineRetailAnalysis").getOrCreate()

In [5]:
# Step 5: Load the Online Retail Dataset CSV into a Spark DataFrame
df = spark.read.csv('online_retail.csv', header=True, inferSchema=True)
# Display the schema to verify correct loading and data types
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [6]:
df.describe().show()

+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|            541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324| 4.61111362608298|15287.690570239585|       NULL|
| stddev|13428.417280803813|16799.737628427752|                NULL|218.08115785023477|96.75985306117967|1713.6003033215982|       NULL|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06|           12346.0|  Australia|
|    max|           C581569|             

In [7]:
df.count()

541909

In [6]:
# Show the first few rows of the DataFrame to understand your data
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [7]:
# Simple Data Analysis Example - Calculate Total Sales per Country; # Assuming 'Quantity' and 'UnitPrice' columns exist, and we're interested in 'Country'
# Add a new column for total sales
df = df.withColumn('TotalSales', col('Quantity') * col('UnitPrice'))
# Show the first few rows of the DataFrame to understand your data
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|        TotalSales|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|15.299999999999999|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|             20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|              22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|             20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|    

In [8]:
# Calculate total sales per country
total_sales_per_country = df.groupBy('Country').agg(sum('TotalSales').alias('TotalSales'))

In [9]:
# Show the results ordered by TotalSales in descending order
total_sales_per_country.orderBy('TotalSales', ascending=False).show()


+---------------+------------------+
|        Country|        TotalSales|
+---------------+------------------+
| United Kingdom|   8187806.3639987|
|    Netherlands| 284661.5399999992|
|           EIRE| 263276.8199999992|
|        Germany|221698.21000000037|
|         France|197403.90000000037|
|      Australia|137077.26999999987|
|    Switzerland| 56385.35000000009|
|          Spain| 54774.58000000016|
|        Belgium|40910.960000000014|
|         Sweden| 36595.90999999998|
|          Japan|35340.619999999995|
|         Norway|35163.460000000014|
|       Portugal|29367.019999999953|
|        Finland|22326.739999999994|
|Channel Islands| 20086.28999999999|
|        Denmark| 18768.13999999999|
|          Italy| 16890.50999999999|
|         Cyprus|12946.289999999994|
|        Austria|10154.319999999996|
|      Hong Kong|10117.039999999997|
+---------------+------------------+
only showing top 20 rows



In [ ]:
# Step 7: Cleanup - Stop the Spark Session
spark.stop()